# Proyecto Backend: Catálogo de Sistemas en un Chip (SoC)

## Contexto
Este proyecto trata de desarrollo de un backend para gestionar un catálogo de Sistemas Embebidos SoC,
implementado paso a paso en un Jupyter Notebook y utilizando una base de datos PostgreSQL remota.

El objetivo es practicar:
- Modelado de bases de datos relacionales
- Conexión remota a bases de datos
- Operaciones CRUD
- Validación de datos
- Organización lógica similar al patrón MVC

---

## Objetivo general
Diseñar e implementar un sistema que permita:
- Registrar Circuitos Integrados de tipo SoC.
- Clasificarlos por fabricante y categoría.
- Asociarlos a interfaces y protocolos de comunicación.
- Consultar, modificar y eliminar información de forma segura y estructurada.

---

## Entidades del sistema
- Fabricantes
- Categorías de Sistemas Embebidos.
- Interfaces de comunicación.
- Protocolos de comunicación.
- Dispositivos SoC.

---

## Estructura del desarrollo
1. Preparación del entorno
2. Elección y creación de la base de datos PostgreSQL
3. Diseño del modelo relacional
4. Creación de tablas
5. Conexión desde Python
6. Operaciones CRUD
7. Validaciones y pruebas
8. Conclusiones

---

## Nota metodológica
El desarrollo se realiza de forma incremental.
Cada sección se implementa y prueba antes de avanzar a la siguiente.


## Prueba de conexión a PostgreSQL desde Jupyter

En esta sección se verifica la conexión remota a la base de datos PostgreSQL
utilizando Python desde Jupyter Notebook.
El objetivo es asegurar que el entorno de desarrollo puede interactuar
correctamente con la base de datos antes de crear el esquema.

In [1]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 161.4 kB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
import psycopg2

DB_HOST = "ep-divine-mouse-ackjw0c7-pooler.sa-east-1.aws.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_RweH6cm2hnLE"
DB_PORT = 5432


In [5]:
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT,
        sslmode="require"
    )
    print("✅ Conexión exitosa a PostgreSQL desde Jupyter")
    conn.close()
except Exception as e:
    print("❌ Error de conexión")
    print(e)


✅ Conexión exitosa a PostgreSQL desde Jupyter


## Creación del esquema de base de datos

En esta sección se crean las tablas del sistema en PostgreSQL.
Las sentencias SQL utilizan `IF NOT EXISTS` para permitir la ejecución
repetida del script sin generar errores.


In [7]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_statements = [

    """
    CREATE TABLE IF NOT EXISTS fabricantes (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        pais VARCHAR(100),
        sitio_web VARCHAR(200)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS categorias_soc (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(100) NOT NULL,
        descripcion TEXT
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS interfaces (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        tipo VARCHAR(50)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS protocolos (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(50) NOT NULL,
        capa VARCHAR(50)
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS soc (
        id SERIAL PRIMARY KEY,
        nombre_modelo VARCHAR(100) NOT NULL,
        fabricante_id INTEGER NOT NULL,
        categoria_id INTEGER NOT NULL,
        arquitectura VARCHAR(50),
        frecuencia_mhz INTEGER,
        descripcion TEXT,
        CONSTRAINT fk_fabricante
            FOREIGN KEY (fabricante_id)
            REFERENCES fabricantes (id)
            ON DELETE RESTRICT,
        CONSTRAINT fk_categoria
            FOREIGN KEY (categoria_id)
            REFERENCES categorias_soc (id)
            ON DELETE RESTRICT
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS soc_interfaces (
        soc_id INTEGER NOT NULL,
        interface_id INTEGER NOT NULL,
        PRIMARY KEY (soc_id, interface_id),
        CONSTRAINT fk_soc
            FOREIGN KEY (soc_id)
            REFERENCES soc (id)
            ON DELETE CASCADE,
        CONSTRAINT fk_interface
            FOREIGN KEY (interface_id)
            REFERENCES interfaces (id)
            ON DELETE CASCADE
    );
    """,

    """
    CREATE TABLE IF NOT EXISTS soc_protocolos (
        soc_id INTEGER NOT NULL,
        protocolo_id INTEGER NOT NULL,
        PRIMARY KEY (soc_id, protocolo_id),
        CONSTRAINT fk_soc_proto
            FOREIGN KEY (soc_id)
            REFERENCES soc (id)
            ON DELETE CASCADE,
        CONSTRAINT fk_protocolo
            FOREIGN KEY (protocolo_id)
            REFERENCES protocolos (id)
            ON DELETE CASCADE
    );
    """
]

for statement in sql_statements:
    cur.execute(statement)

conn.commit()
cur.close()
conn.close()

print("✅ Tablas creadas correctamente (o ya existían)")


✅ Tablas creadas correctamente (o ya existían)


## Inserción de datos de prueba

En esta sección se insertan datos mínimos de prueba en las tablas del sistema.
Las inserciones están diseñadas para evitar duplicados, permitiendo ejecutar
el script múltiples veces sin generar inconsistencias.


In [8]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_inserts = [

    # Fabricantes
    """
    INSERT INTO fabricantes (nombre, pais, sitio_web)
    SELECT 'Espressif', 'China', 'https://www.espressif.com'
    WHERE NOT EXISTS (
        SELECT 1 FROM fabricantes WHERE nombre = 'Espressif'
    );
    """,

    """
    INSERT INTO fabricantes (nombre, pais, sitio_web)
    SELECT 'STMicroelectronics', 'Suiza', 'https://www.st.com'
    WHERE NOT EXISTS (
        SELECT 1 FROM fabricantes WHERE nombre = 'STMicroelectronics'
    );
    """,

    # Categorías SoC
    """
    INSERT INTO categorias_soc (nombre, descripcion)
    SELECT 'Microcontrolador', 'SoC orientado a control y bajo consumo'
    WHERE NOT EXISTS (
        SELECT 1 FROM categorias_soc WHERE nombre = 'Microcontrolador'
    );
    """,

    """
    INSERT INTO categorias_soc (nombre, descripcion)
    SELECT 'SoC IoT', 'SoC con conectividad integrada para IoT'
    WHERE NOT EXISTS (
        SELECT 1 FROM categorias_soc WHERE nombre = 'SoC IoT'
    );
    """,

    # Interfaces
    """
    INSERT INTO interfaces (nombre, tipo)
    SELECT 'UART', 'Serial'
    WHERE NOT EXISTS (
        SELECT 1 FROM interfaces WHERE nombre = 'UART'
    );
    """,

    """
    INSERT INTO interfaces (nombre, tipo)
    SELECT 'SPI', 'Serial'
    WHERE NOT EXISTS (
        SELECT 1 FROM interfaces WHERE nombre = 'SPI'
    );
    """,

    # Protocolos
    """
    INSERT INTO protocolos (nombre, capa)
    SELECT 'TCP/IP', 'Red'
    WHERE NOT EXISTS (
        SELECT 1 FROM protocolos WHERE nombre = 'TCP/IP'
    );
    """,

    """
    INSERT INTO protocolos (nombre, capa)
    SELECT 'MQTT', 'Aplicación'
    WHERE NOT EXISTS (
        SELECT 1 FROM protocolos WHERE nombre = 'MQTT'
    );
    """
]

for statement in sql_inserts:
    cur.execute(statement)

conn.commit()
cur.close()
conn.close()

print("✅ Datos de prueba insertados correctamente (o ya existían)")


✅ Datos de prueba insertados correctamente (o ya existían)


## Operaciones CRUD básicas: creación y listado de SoC

En esta sección se implementan las primeras operaciones CRUD reales del sistema:
- Inserción controlada de un SoC
- Consulta y listado de SoC con información relacionada

Estas operaciones se realizan directamente desde Jupyter Notebook,
simulando la lógica de un backend.


In [9]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_insert_soc = """
INSERT INTO soc (
    nombre_modelo,
    fabricante_id,
    categoria_id,
    arquitectura,
    frecuencia_mhz,
    descripcion
)
SELECT
    'ESP32',
    f.id,
    c.id,
    'Xtensa',
    240,
    'SoC con WiFi y Bluetooth integrado'
FROM fabricantes f, categorias_soc c
WHERE f.nombre = 'Espressif'
  AND c.nombre = 'SoC IoT'
  AND NOT EXISTS (
      SELECT 1 FROM soc WHERE nombre_modelo = 'ESP32'
  );
"""

cur.execute(sql_insert_soc)
conn.commit()

cur.close()
conn.close()

print("✅ SoC insertado correctamente (o ya existía)")


✅ SoC insertado correctamente (o ya existía)


In [11]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_select_soc = """
SELECT
    s.id,
    s.nombre_modelo,
    f.nombre AS fabricante,
    c.nombre AS categoria,
    s.arquitectura,
    s.frecuencia_mhz
FROM soc s
JOIN fabricantes f ON s.fabricante_id = f.id
JOIN categorias_soc c ON s.categoria_id = c.id
ORDER BY s.id;
"""

cur.execute(sql_select_soc)
rows = cur.fetchall()

cur.close()
conn.close()

for row in rows:
    print(row)


(1, 'ESP32', 'Espressif', 'SoC IoT', 'Xtensa', 240)


## Asociación de Interfaces y Protocolos a un SoC

En esta sección se implementan las relaciones de muchos a muchos (N:M)
entre los SoC y las interfaces y protocolos que soportan.
Las inserciones evitan duplicados y mantienen la integridad referencial.


In [12]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_insert_interfaces = """
INSERT INTO soc_interfaces (soc_id, interface_id)
SELECT s.id, i.id
FROM soc s, interfaces i
WHERE s.nombre_modelo = 'ESP32'
  AND i.nombre IN ('UART', 'SPI')
  AND NOT EXISTS (
      SELECT 1 FROM soc_interfaces si
      WHERE si.soc_id = s.id
        AND si.interface_id = i.id
  );
"""

cur.execute(sql_insert_interfaces)
conn.commit()

cur.close()
conn.close()

print("✅ Interfaces asociadas correctamente (o ya existían)")


✅ Interfaces asociadas correctamente (o ya existían)


In [13]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_insert_protocolos = """
INSERT INTO soc_protocolos (soc_id, protocolo_id)
SELECT s.id, p.id
FROM soc s, protocolos p
WHERE s.nombre_modelo = 'ESP32'
  AND p.nombre IN ('TCP/IP', 'MQTT')
  AND NOT EXISTS (
      SELECT 1 FROM soc_protocolos sp
      WHERE sp.soc_id = s.id
        AND sp.protocolo_id = p.id
  );
"""

cur.execute(sql_insert_protocolos)
conn.commit()

cur.close()
conn.close()

print("✅ Protocolos asociados correctamente (o ya existían)")


✅ Protocolos asociados correctamente (o ya existían)


In [14]:
import psycopg2

conn = psycopg2.connect(
    host=DB_HOST,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT,
    sslmode="require"
)

cur = conn.cursor()

sql_full_view = """
SELECT
    s.nombre_modelo,
    f.nombre AS fabricante,
    c.nombre AS categoria,
    STRING_AGG(DISTINCT i.nombre, ', ') AS interfaces,
    STRING_AGG(DISTINCT p.nombre, ', ') AS protocolos
FROM soc s
JOIN fabricantes f ON s.fabricante_id = f.id
JOIN categorias_soc c ON s.categoria_id = c.id
LEFT JOIN soc_interfaces si ON si.soc_id = s.id
LEFT JOIN interfaces i ON si.interface_id = i.id
LEFT JOIN soc_protocolos sp ON sp.soc_id = s.id
LEFT JOIN protocolos p ON sp.protocolo_id = p.id
GROUP BY s.nombre_modelo, f.nombre, c.nombre;
"""

cur.execute(sql_full_view)
rows = cur.fetchall()

cur.close()
conn.close()

for row in rows:
    print(row)


('ESP32', 'Espressif', 'SoC IoT', 'SPI, UART', 'MQTT, TCP/IP')


## Encapsulación de la lógica de acceso a datos

En esta sección se refactoriza el código existente,
encapsulando la lógica de acceso a la base de datos en funciones.
Esto simula la capa Modelo dentro de una arquitectura tipo MVC.


In [15]:
import psycopg2

def get_connection():
    return psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT,
        sslmode="require"
    )


In [16]:
def create_soc(nombre_modelo, fabricante, categoria, arquitectura, frecuencia_mhz, descripcion):
    conn = get_connection()
    cur = conn.cursor()

    sql = """
    INSERT INTO soc (
        nombre_modelo,
        fabricante_id,
        categoria_id,
        arquitectura,
        frecuencia_mhz,
        descripcion
    )
    SELECT
        %s,
        f.id,
        c.id,
        %s,
        %s,
        %s
    FROM fabricantes f, categorias_soc c
    WHERE f.nombre = %s
      AND c.nombre = %s
      AND NOT EXISTS (
          SELECT 1 FROM soc WHERE nombre_modelo = %s
      );
    """

    cur.execute(sql, (
        nombre_modelo,
        arquitectura,
        frecuencia_mhz,
        descripcion,
        fabricante,
        categoria,
        nombre_modelo
    ))

    conn.commit()
    cur.close()
    conn.close()


In [17]:
def list_soc():
    conn = get_connection()
    cur = conn.cursor()

    sql = """
    SELECT
        s.id,
        s.nombre_modelo,
        f.nombre AS fabricante,
        c.nombre AS categoria,
        s.arquitectura,
        s.frecuencia_mhz
    FROM soc s
    JOIN fabricantes f ON s.fabricante_id = f.id
    JOIN categorias_soc c ON s.categoria_id = c.id
    ORDER BY s.id;
    """

    cur.execute(sql)
    rows = cur.fetchall()

    cur.close()
    conn.close()
    return rows


In [18]:
create_soc(
    nombre_modelo="STM32F4",
    fabricante="STMicroelectronics",
    categoria="Microcontrolador",
    arquitectura="ARM Cortex-M4",
    frecuencia_mhz=168,
    descripcion="SoC orientado a aplicaciones industriales"
)

for soc in list_soc():
    print(soc)


(1, 'ESP32', 'Espressif', 'SoC IoT', 'Xtensa', 240)
(2, 'STM32F4', 'STMicroelectronics', 'Microcontrolador', 'ARM Cortex-M4', 168)


## Validaciones y manejo de errores

En esta sección se incorporan validaciones básicas y manejo de errores
en la capa de acceso a datos, con el objetivo de garantizar la integridad
de la información y la robustez del sistema.


In [19]:
def create_soc(nombre_modelo, fabricante, categoria, arquitectura, frecuencia_mhz, descripcion):
    if not nombre_modelo or not fabricante or not categoria:
        print("❌ Error: nombre_modelo, fabricante y categoria son obligatorios")
        return

    if frecuencia_mhz is not None and frecuencia_mhz <= 0:
        print("❌ Error: la frecuencia debe ser un número positivo")
        return

    try:
        conn = get_connection()
        cur = conn.cursor()

        sql = """
        INSERT INTO soc (
            nombre_modelo,
            fabricante_id,
            categoria_id,
            arquitectura,
            frecuencia_mhz,
            descripcion
        )
        SELECT
            %s,
            f.id,
            c.id,
            %s,
            %s,
            %s
        FROM fabricantes f, categorias_soc c
        WHERE f.nombre = %s
          AND c.nombre = %s
          AND NOT EXISTS (
              SELECT 1 FROM soc WHERE nombre_modelo = %s
          );
        """

        cur.execute(sql, (
            nombre_modelo,
            arquitectura,
            frecuencia_mhz,
            descripcion,
            fabricante,
            categoria,
            nombre_modelo
        ))

        if cur.rowcount == 0:
            print("⚠️ No se insertó el SoC (puede existir ya o no existir el fabricante/categoría)")
        else:
            print("✅ SoC creado correctamente")

        conn.commit()
        cur.close()
        conn.close()

    except Exception as e:
        print("❌ Error al crear SoC")
        print(e)


In [20]:
# Caso correcto
create_soc(
    nombre_modelo="RP2040",
    fabricante="Raspberry Pi",
    categoria="Microcontrolador",
    arquitectura="ARM Cortex-M0+",
    frecuencia_mhz=133,
    descripcion="SoC dual-core utilizado en Raspberry Pi Pico"
)

# Caso inválido (frecuencia negativa)
create_soc(
    nombre_modelo="TEST_BAD",
    fabricante="Espressif",
    categoria="SoC IoT",
    arquitectura="Xtensa",
    frecuencia_mhz=-10,
    descripcion="Error de prueba"
)


⚠️ No se insertó el SoC (puede existir ya o no existir el fabricante/categoría)
❌ Error: la frecuencia debe ser un número positivo
